In [1]:
!pip install requests

In [1]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
# goals
# - line history
# - % bets
# - % money
# NFL side and total

In [2]:
# scraping function
def scrape_for_soup(base_url, url):
    # send GET request
    response = requests.get(base_url+url)

    # status check
    if response.status_code != 200:
        return(f"Error: {response.status_code}")

    # parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [3]:
base_url = 'https://www.sportsbookreview.com'
url = '/betting-odds/nfl-football/'
soup = scrape_for_soup(base_url, url)

# parse links for game spread and spread history
games = soup.find_all('div', id='tbody-nfl')
main_links = set()
line_links = set()
for game in games:
    for a_tag in game.find_all('a', href=True):
        link = a_tag['href']
        if 'scores' in link:
            main_links.add(link)
        if 'line-history' in link:
            line_links.add(link)

In [4]:
main_links

{'/scores/nfl-football/matchup/304841/',
 '/scores/nfl-football/matchup/305068/',
 '/scores/nfl-football/matchup/305069/',
 '/scores/nfl-football/matchup/305070/',
 '/scores/nfl-football/matchup/305074/',
 '/scores/nfl-football/matchup/305078/',
 '/scores/nfl-football/matchup/305079/',
 '/scores/nfl-football/matchup/305080/',
 '/scores/nfl-football/matchup/305081/',
 '/scores/nfl-football/matchup/305083/',
 '/scores/nfl-football/matchup/305088/',
 '/scores/nfl-football/matchup/305093/',
 '/scores/nfl-football/matchup/305105/',
 '/scores/nfl-football/matchup/305112/'}

In [4]:
# iterate through each link to scrape target data
# send request
game_url = list(main_links)[0]
game_soup = scrape_for_soup(base_url, game_url)
# pull number bets on spread and o/u

In [7]:
print(base_url+game_url)

https://www.sportsbookreview.com/scores/nfl-football/matchup/305134/


In [6]:
# spread bets
spread = game_soup.find('div', text=re.compile('Spread'))
home_perc = spread.next_sibling.get_text()[:-1]
away_perc = spread.previous_sibling.get_text()[:-1]

# over/under bets
total = game_soup.find('div', text=re.compile('Total'))
over_perc = total.previous_sibling.get_text()[:-3]
under_perc = total.next_sibling.get_text()[:-3]

# teams and spread
away_div = game_soup.find('div', class_=re.compile('TeamName'))
home_div = game_soup.find('div', class_=re.compile('TeamName')).findNext('div', class_=re.compile('TeamName'))
home_team = re.findall('.+?(?=\+|-|PK)',home_div.get_text())[0]
away_team = re.findall('.+?(?=\+|-|PK)',away_div.get_text())[0]
away_spread = away_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
home_spread = home_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
print(home_team, home_spread)
print(away_team, away_spread)
print(home_team + ' ' + home_spread + ' % bets: ' + home_perc)
print(away_team + ' ' + away_spread + ' % bets: ' + away_perc)
print('% Over bets: ' + over_perc)
print('% Under bets: ' + under_perc)

BaltimoreRavens -6.5
WashingtonCommanders +6.5
BaltimoreRavens -6.5 % bets: 37
WashingtonCommanders +6.5 % bets: 63
% Over bets: 65
% Under bets: 35


In [18]:
# pull total
game_soup.find('div', text=re.compile('[1-9]\d{1}(\.\d+)?'))

<div class="fs-8 fw-bold my-3" id="gameDate">Sunday, October 13, 2024 - 1:00 PM EDT</div>

In [ ]:
# next: pull total
line_url = list(line_links)[0]
line_soup = scrape_for_soup(base_url, line_url)

In [16]:
print(base_url+line_url)

https://www.sportsbookreview.com/betting-odds/nfl-football/line-history/305142/


In [8]:
div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
print(div)

<div class="row GameMatchup_mainColumn__cILb0"><aside class="col-12 col-lg-3"><div class="mt-2"><h2 class="d-flex align-items-center justify-content-between mb-0 py-1 px-3 bg-dark h4" data-bs-theme="dark">Betting Consensus</h2><div class="d-flex justify-content-center pt-2"><div class="col-4 d-flex flex-column justify-content-center text-center text-primary fx-bold pb-3 GameMatchup_TeamName__hOY6B"><p class="h4">Las Vegas<br/>Raiders</p><span class="h5 mb-2 text-secondary opacity-75">+2.5</span></div><div class="d-flex justify-content-center align-items-center mx-2 mt-4 rounded-circle fs-7 text-white bg-dark text-center GameMatchup_TagCircle__FABto">vs</div><div class="col-4 d-flex flex-column justify-content-center text-center text-primary fx-bold pb-3 GameMatchup_TeamName__hOY6B"><p class="h4">Denver<br/>Broncos</p><span class="h5 mb-2 text-secondary opacity-75">-2.5</span></div></div><div class="d-flex flex-column w-100 border-top undefined Rows_Rows__wF9TM"><div class="d-flex align